In [4]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import chardet

def detect_file_encoding(file_path):
    """Detects the encoding of a file."""
    with open(file_path, 'rb') as f:
        raw_data = f.read(10000)  # Read a sample of the file
    result = chardet.detect(raw_data)
    return result['encoding']

def load_ground_truth(file_path):
    """Load the ground truth data from an Excel file."""
    return pd.read_excel(file_path)

def try_open_file(file_path, encodings=['utf-8', 'latin-1', 'iso-8859-1']):
    """Tries to open a file with multiple encodings."""
    for encoding in encodings:
        try:
            df = pd.read_csv(file_path, sep=';', encoding=encoding, on_bad_lines='warn')
            return df, encoding
        except UnicodeDecodeError:
            continue
    raise UnicodeDecodeError(f"Unable to decode file with available encodings: {file_path}")

def load_predicted_dataframes(folder_path):
    """Load all CSV files from a folder and return a dictionary of DataFrames."""
    dataframes = {}
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.csv'):
            try:
                df, used_encoding = try_open_file(os.path.join(folder_path, file_name))
                dataframes[file_name] = df
                print(f"Loaded {file_name} with encoding {used_encoding}")
            except Exception as e:
                print(f"Failed to load {file_name}: {e}")
    return dataframes

def clean_dataframe(df, valid_classes):
    """Clean the DataFrame by excluding invalid or empty classifications."""
    initial_rows = df.shape[0]
    
    if 'classification_cleaned' in df.columns:
        df = df[df['classification_cleaned'].isin(valid_classes) & df['classification_cleaned'].notna()]
        discarded_rows = initial_rows - df.shape[0]
    else:
        discarded_rows = initial_rows
    
    return df, discarded_rows

def calculate_metrics(y_true, y_pred):
    """Calculate accuracy, precision, recall, and F1 score (both weighted and macro)."""
    metrics = {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision_macro': precision_score(y_true, y_pred, average='macro'),
        'recall_macro': recall_score(y_true, y_pred, average='macro'),
        'f1_macro': f1_score(y_true, y_pred, average='macro'),
        'precision_weighted': precision_score(y_true, y_pred, average='weighted'),
        'recall_weighted': recall_score(y_true, y_pred, average='weighted'),
        'f1_weighted': f1_score(y_true, y_pred, average='weighted')
    }
    return metrics

def calculate_class_distribution(series):
    """Calculate class distribution for a series of classifications."""
    distribution = (series.value_counts(normalize=True) * 100).round(2).to_dict()
    return distribution

def extract_task_from_filename(filename):
    """Extract task from filename string."""
    start = filename.find('_EN_') + len('_EN_')
    end = filename.find('_classify_replies')
    return filename[start:end]

def process_files(ground_truth_file, predicted_folder, valid_classes):
    ground_truth_df = load_ground_truth(ground_truth_file)
    predicted_dfs = load_predicted_dataframes(predicted_folder)

    # Mapping for new class names
    class_mapping = {
        "0. appropriate": "appropriate",
        "1. inappropriate": "inappropriate",
        "2. offensive": "offensive",
        "3. violent": "violent"
    }

    # Rename classes in type_reply
    ground_truth_df["type_reply"] = ground_truth_df["type_reply"].map(class_mapping)
    
    y_true = ground_truth_df['type_reply']
    results = []

    total_discarded_rows = 0
    skipped_files = []
    rows_used_dict = {}
    rows_not_used_dict = {}
    rows_not_used_per_file = {}
    rows_not_used_contributions = []

    common_index = y_true.index

    for file_name, df_pred in predicted_dfs.items():
        if 'classification_cleaned' not in df_pred.columns:
            skipped_files.append(file_name)
            continue
        try:
            df_pred_clean, discarded_rows = clean_dataframe(df_pred, valid_classes)
            new_common_index = common_index.intersection(df_pred_clean.index)
            unused_rows_contribution = len(common_index) - len(new_common_index)
            rows_not_used_contributions.append((file_name, unused_rows_contribution))
            common_index = new_common_index
            total_discarded_rows += discarded_rows
        except Exception as e:
            print(f"Failed to process {file_name}: {e}")
            skipped_files.append(file_name)

    # Remove rows with 'none' classification in any file
    for file_name, df_pred in predicted_dfs.items():
        if 'classification_cleaned' in df_pred.columns:
            df_pred_clean = df_pred.loc[common_index]
            if 'none' in df_pred_clean['classification_cleaned'].values:
                common_index = common_index.difference(df_pred_clean.index[df_pred_clean['classification_cleaned'] == 'none'])

    # Process remaining files with common index
    for file_name, df_pred in predicted_dfs.items():
        if 'classification_cleaned' not in df_pred.columns or file_name in skipped_files:
            continue
        try:
            df_pred_clean = df_pred.loc[common_index]
            y_pred = df_pred_clean['classification_cleaned']
            y_true_common = y_true.loc[common_index]
            
            metrics = calculate_metrics(y_true_common, y_pred)
            metrics['file_name'] = file_name

            # Extract task from filename
            metrics['task'] = extract_task_from_filename(file_name)

            # Calculate class distributions
            ground_truth_distribution = calculate_class_distribution(y_true_common)
            predicted_distribution = calculate_class_distribution(y_pred)
            metrics['ground_truth_distribution'] = str(ground_truth_distribution)
            metrics['predicted_distribution'] = str(predicted_distribution)

            # Record used and not used row indices
            rows_used = common_index.tolist()
            rows_not_used = [idx for idx in df_pred.index if idx not in common_index]

            metrics['rows_used'] = rows_used
            metrics['rows_not_used'] = rows_not_used
            metrics['count_rows_used'] = len(rows_used)
            metrics['count_rows_not_used'] = len(rows_not_used)
            rows_not_used_per_file[file_name] = len(rows_not_used)

            results.append(metrics)
        except Exception as e:
            print(f"Failed to process {file_name}: {e}")
            skipped_files.append(file_name)

    metrics_df = pd.DataFrame(results)
    metrics_df['rows_not_used_due_to_file'] = metrics_df['file_name'].map(dict(rows_not_used_contributions))

    if skipped_files:
        print("\nSkipped files:")
        for file_name in skipped_files:
            print(file_name)

    return metrics_df, total_discarded_rows


valid_classes = ['appropriate', 'inappropriate', 'offensive', 'violent']
ground_truth_file = r'C:\Users\aboeck\Documents\ICMT\MASTERARBEIT_DI\backup_03_07\EVALUATION\FINAL_IMSyPP_EN_561_PREPROCESSED_no_restricted_videos_features_2ktoken_w_prompt_u_desc_audio_object_with_video_and_reply_metadata_no_missing.xlsx'
predicted_folder = r"./"

metrics_df, total_discarded_rows = process_files(ground_truth_file, predicted_folder, valid_classes)
print(metrics_df)
print(f"Total discarded rows: {total_discarded_rows}")

# Save the metrics DataFrame as an Excel file
output_file = 'metrics_results_LLAMA3_NEWEST.xlsx'
metrics_df.to_excel(output_file, index=False)
print(f"Metrics saved to {output_file}")


Loaded _EN_reply_classify_replies_561_LLAMA3_Hatebase_FINAL_AAAAAAAABSOLUTE_here.csv with encoding utf-8
Loaded _EN_reply_classify_replies_561_LLAMA3_Hatebase_FINAL_AAAAAAAABSOLUTE_here_new_THIRD-new21.csv with encoding utf-8
Loaded _EN_reply_classify_replies_561_LLAMA3_Hatebase_FINAL_FINAL_SECOND.csv with encoding utf-8
Loaded _EN_reply_comment_classify_replies_561_LLAMA3_Hatebase_FINAL.csv with encoding utf-8
Loaded _EN_reply_comment_classify_replies_561_LLAMA3_Hatebase_FINAL_AAAAAAAABSOLUTEeeeeee.csv with encoding utf-8
Loaded _EN_reply_comment_classify_replies_561_LLAMA3_Hatebase_FINAL_AAAAAAAABSOLUTEeeeeee_SECOND.csv with encoding utf-8
Loaded _EN_reply_comment_classify_replies_561_LLAMA3_Hatebase_Letzter_Rest.csv with encoding utf-8
Loaded _EN_reply_comment_title_classify_replies_561_LLAMA3_Hatebase_FINAL_AAAAAAAABSOLUTEeeeeee_SECOND_new.csv with encoding utf-8
Loaded _EN_reply_comment_title_classify_replies_561_LLAMA3_Hatebase_FINAL_AAAAAAAABSOLUTE_here_new_THIRD-new.csv with en

c:\Users\aboeck\AppData\Local\anaconda3\envs\new_MA\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\aboeck\AppData\Local\anaconda3\envs\new_MA\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


    accuracy  precision_macro  recall_macro  f1_macro  precision_weighted  \
0   0.679912         0.489530      0.752195  0.387320            0.879333   
1   0.677704         0.468852      0.741395  0.357333            0.875031   
2   0.675497         0.483747      0.744029  0.375226            0.882424   
3   0.746137         0.401190      0.509546  0.294280            0.789490   
4   0.664459         0.585761      0.747122  0.468470            0.905431   
5   0.664459         0.505529      0.748802  0.403860            0.888270   
6   0.655629         0.483747      0.742545  0.394737            0.866399   
7   0.637969         0.486484      0.736747  0.391672            0.876497   
8   0.646799         0.518537      0.743004  0.424446            0.883757   
9   0.637969         0.565772      0.740106  0.463668            0.883737   
10  0.644592         0.500920      0.737242  0.390999            0.888560   
11  0.721854         0.441926      0.742127  0.449974            0.845725   